In [2]:
# https://pypi.org/project/PuLP/
# https://coin-or.github.io/pulp/guides/index.html

In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import os
import json
import pandas as pd
from sqlite3 import connect

In [6]:
pd.options.display.float_format = "{:.2f}".format

In [7]:
# Get the absolute path of the file within the INPUTS directory
file_path = os.path.join(os.getcwd(), 'INPUTS', 'case_study_data.csv')

# Print the file path
print(file_path)

C:\Users\KonuTech\PycharmProjects\supply-chain-case-study\INPUTS\case_study_data.csv


In [8]:
# Get the absolute path of the config within the CONFIGS directory
config_path = os.path.join(os.getcwd(), 'CONFIGS', 'config.json')

# Print the fileconfig_pathpath
print(config_path)

C:\Users\KonuTech\PycharmProjects\supply-chain-case-study\CONFIGS\config.json


In [9]:
with open(config_path, encoding='utf-8') as f:
    CONFIG = json.load(f)

In [10]:
INDEX_COL = CONFIG["INPUTS"]["INDEX_COLUMNS"]
SEP = CONFIG["INPUTS"]["SEPARATOR"]
DECIMAL = CONFIG["INPUTS"]["DECIMAL"]
ENCODING = CONFIG["INPUTS"]["ENCODING"]
DATE_COLUMNS = CONFIG["INPUTS"]["DATE_COLUMNS"]
FLOAT_PRECISION = CONFIG["INPUTS"]["FLOAT_PRECISION"]
DTYPE = CONFIG["INPUTS"]["DTYPE"]
COLUMNS_WITH_NAN_VALUES = CONFIG["INPUTS"]["COLUMNS_WITH_NAN_VALUES"]

In [11]:
df = pd.read_csv(
    file_path,
#     index_col=INDEX_COL,
    sep=SEP,
    encoding=ENCODING,
    engine="c",
    low_memory=False,
    parse_dates=DATE_COLUMNS,
    dtype=DTYPE
)

# PREPROCESSING

In [12]:
# Use regex to separate the string parts of the Order ID column into separate fields
df[['Order ID prefix', 'Order ID year', 'Order ID number']] = df['Order ID'].str.extract(r'(\w+)-(\d+)-(\d+)')

# Use regex to separate the string parts of the Order Date column into separate fields
df[['Order Year', 'Order Month', 'Order Day']] = pd.to_datetime(df['Order Date']).dt.strftime('%Y-%m-%d').str.extract(r'(\d{4})-(\d{2})-(\d{2})')

# SQL

In [13]:
conn = connect(':memory:')
df.to_sql('data', conn)

9993

In [14]:
pd.read_sql(
"""
    SELECT
        *
    FROM data
""",
conn
)

,index,Order ID,Order Date,Ship Mode,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Order ID prefix,Order ID year,Order ID number,Order Year,Order Month,Order Day
0,0,PL-2015-110870,2015-12-12 00:00:00,First Class,West,TEC-AC-10002926,Technology,Accessories,Logitech Wireless Marathon Mouse M705,299.94,6,0.00,NaN,PL,2015,110870,2015,12,12
1,1,PL-2015-110870,2015-12-12 00:00:00,First Class,West,OFF-SU-10001225,Office Supplies,Supplies,Staple remover,25.76,7,0.00,NaN,PL,2015,110870,2015,12,12
2,2,PL-2014-143210,2014-12-01 00:00:00,First Class,East,TEC-PH-10004434,Technology,Phones,Cisco IP Phone 7961G VoIP phone - Dark gray,271.90,2,0.00,NaN,PL,2014,143210,2014,12,01
3,3,PL-2014-143210,2014-12-01 00:00:00,First Class,East,FUR-FU-10003878,Furniture,Furnishings,"Linden 10"" Round Wall Clock, Black",45.84,3,0.00,NaN,PL,2014,143210,2014,12,01
4,4,PL-2014-143210,2014-12-01 00:00:00,First Class,East,FUR-FU-10002268,Furniture,Furnishings,Ultra Door Push Plate,9.82,2,0.00,NaN,PL,2014,143210,2014,12,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,9988,SL-2015-165743,2015-11-20 00:00:00,Second Class,West,OFF-BI-10001758,Office Supplies,Binders,Wilson Jones 14 Line Acrylic Coated Pressboard Data Binders,9.61,6,0.70,-7.37,SL,2015,165743,2015,11,20
9989,9989,SL-2017-105998,2017-11-03 00:00:00,First Class,West,TEC-AC-10004469,Technology,Accessories,Microsoft Sculpt Comfort Mouse,199.75,5,0.00,87.89,SL,2017,105998,2017,11,03
9990,9990,SL-2017-105998,2017-11-03 00:00:00,First Class,West,FUR-TA-10001095,Furniture,Tables,Chromcraft Round Conference Tables,1673.18,12,0.20,20.91,SL,2017,105998,2017,11,03
9991,9991,SL-2014-148194,2014-05-04 00:00:00,First Class,West,FUR-FU-10001852,Furniture,Furnishings,"Eldon Regeneration Recycled Desk Accessories, Smoke",12.18,7,0.00,3.90,SL,2014,148194,2014,05,04


In [15]:
pd.read_sql(
"""
    SELECT
        "Order ID prefix"
        ,Region
        ,"Ship Mode"
        ,Category
        ,"Sub-Category"
        ,"Order Year"
        ,COUNT("Order ID") AS Count
        ,AVG("Profit") AS average
    FROM data
    WHERE
        "Order ID prefix" IN (
            'SL'
            --,'PL'
        )
    GROUP BY
        "Order ID prefix"
        ,Region
        ,"Ship Mode"
        ,Category
        ,"Sub-Category"
    --HAVING average <= 0
    ORDER BY average ASC
""",
conn
)

,Order ID prefix,Region,Ship Mode,Category,Sub-Category,Order Year,Count,average
0,SL,Central,First Class,Office Supplies,Envelopes,2017,1,NaN
1,SL,Central,Second Class,Office Supplies,Fasteners,2014,1,NaN
2,SL,East,First Class,Office Supplies,Appliances,2015,2,NaN
3,SL,East,Same Day,Office Supplies,Labels,2016,1,NaN
4,SL,East,Same Day,Technology,Machines,2014,1,NaN
...,...,...,...,...,...,...,...,...
220,SL,South,Second Class,Technology,Copiers,2017,1,629.99
221,SL,Central,Standard Class,Technology,Copiers,2015,3,722.49
222,SL,West,Same Day,Technology,Copiers,2017,1,839.99
223,SL,East,Standard Class,Technology,Copiers,2015,3,1225.64


In [16]:
pd.read_sql(
"""
SELECT SUM(Count) FROM (
    SELECT
        "Order ID prefix"
        ,Region
        ,"Ship Mode"
        ,Category
        ,"Sub-Category"
        ,"Order Year"
        ,COUNT("Order ID") AS Count
        ,AVG("Profit") AS average
    FROM data
    WHERE
        "Order ID prefix" IN (
            'SL'
            --,'PL'
        )
    GROUP BY
        "Order ID prefix"
        ,Region
        ,"Ship Mode"
        ,Category
        ,"Sub-Category"
    --HAVING average <= 0
    ORDER BY average ASC
    )
""",
conn
)

,SUM(Count)
0,1685


In [17]:
pd.read_sql(
"""
    SELECT
        "Order ID prefix"
        ,Region
        ,"Ship Mode"
        ,Category
        ,"Sub-Category"
        ,"Order Year"
        ,COUNT("Order ID") AS Count
        ,AVG("Profit") AS average
    FROM data
    WHERE
        "Order ID prefix" IN (
            'SL'
            --,'PL'
        )
    GROUP BY
        "Order ID prefix"
        ,Region
        ,"Ship Mode"
        ,Category
        ,"Sub-Category"
    HAVING average <= 0
    ORDER BY average ASC
""",
conn
)

,Order ID prefix,Region,Ship Mode,Category,Sub-Category,Order Year,Count,average
0,SL,South,Same Day,Technology,Machines,2017,1,-3839.99
1,SL,East,First Class,Technology,Machines,2017,1,-935.96
2,SL,East,Second Class,Technology,Machines,2016,1,-643.71
3,SL,South,Standard Class,Technology,Machines,2014,1,-572.00
4,SL,Central,Second Class,Furniture,Tables,2017,1,-566.56
...,...,...,...,...,...,...,...,...
52,SL,East,First Class,Office Supplies,Supplies,2015,3,-2.19
53,SL,East,First Class,Office Supplies,Storage,2017,8,-2.17
54,SL,East,Second Class,Office Supplies,Supplies,2015,2,-1.31
55,SL,South,Second Class,Office Supplies,Storage,2014,5,-0.85


In [18]:
pd.read_sql(
"""
    SELECT SUM(Count) FROM (
    SELECT
        "Order ID prefix"
        ,Region
        ,"Ship Mode"
        ,Category
        ,"Sub-Category"
        ,"Order Year"
        ,COUNT("Order ID") AS Count
        ,AVG("Profit") AS average
    FROM data
    WHERE
        "Order ID prefix" IN (
            'SL'
            --,'PL'
        )
    GROUP BY
        "Order ID prefix"
        ,Region
        ,"Ship Mode"
        ,Category
        ,"Sub-Category"
    HAVING average <= 0
    ORDER BY average ASC
    )
""",
conn
)

,SUM(Count)
0,450


In [19]:
pd.read_sql(
"""
    SELECT
        "Order ID prefix"
        ,Region
        ,"Ship Mode"
        ,Category
        ,"Sub-Category"
        ,"Order Year"
        ,COUNT("Order ID") AS Count
        ,AVG("Profit") AS average
    FROM data
    WHERE
        "Order ID prefix" IN (
            --'SL'
            'PL'
        )
    GROUP BY
        "Order ID prefix"
        ,Region
        ,"Ship Mode"
        ,Category
        ,"Sub-Category"
    --HAVING average <= 0
    ORDER BY average ASC
""",
conn
)

,Order ID prefix,Region,Ship Mode,Category,Sub-Category,Order Year,Count,average
0,PL,Central,First Class,Office Supplies,Supplies,2017,2,NaN
1,PL,South,Same Day,Furniture,Chairs,2014,1,NaN
2,PL,South,Same Day,Furniture,Tables,2016,1,NaN
3,PL,South,Second Class,Technology,Machines,2016,2,NaN
4,PL,West,Same Day,Technology,Copiers,2016,1,NaN
...,...,...,...,...,...,...,...,...
260,PL,South,Second Class,Technology,Copiers,2017,2,764.99
261,PL,West,First Class,Technology,Copiers,2017,5,828.36
262,PL,East,First Class,Technology,Copiers,2014,5,1089.25
263,PL,East,Standard Class,Technology,Copiers,2014,8,1403.59


In [20]:
pd.read_sql(
"""
    SELECT SUM(Count) FROM (
    SELECT
        "Order ID prefix"
        ,Region
        ,"Ship Mode"
        ,Category
        ,"Sub-Category"
        ,"Order Year"
        ,COUNT("Order ID") AS Count
        ,AVG("Profit") AS average
    FROM data
    WHERE
        "Order ID prefix" IN (
            --'SL'
            'PL'
        )
    GROUP BY
        "Order ID prefix"
        ,Region
        ,"Ship Mode"
        ,Category
        ,"Sub-Category"
    --HAVING average <= 0
    ORDER BY average ASC
    )
""",
conn
)

,SUM(Count)
0,8308


In [21]:
pd.read_sql(
"""
    SELECT
        "Order ID prefix"
        ,Region
        ,"Ship Mode"
        ,Category
        ,"Sub-Category"
        ,"Order Year"
        ,COUNT("Order ID") AS Count
        ,AVG("Profit") AS average
    FROM data
    WHERE
        "Order ID prefix" IN (
            --'SL'
            'PL'
        )
    GROUP BY
        "Order ID prefix"
        ,Region
        ,"Ship Mode"
        ,Category
        ,"Sub-Category"
    HAVING average <= 0
    ORDER BY average ASC
""",
conn
)

,Order ID prefix,Region,Ship Mode,Category,Sub-Category,Order Year,Count,average
0,PL,East,First Class,Technology,Machines,2016,7,-402.26
1,PL,Central,Standard Class,Technology,Machines,2014,11,-175.10
2,PL,South,Standard Class,Technology,Machines,2016,9,-135.49
3,PL,East,Second Class,Furniture,Tables,2016,11,-134.69
4,PL,Central,Second Class,Furniture,Bookcases,2016,8,-119.54
...,...,...,...,...,...,...,...,...
27,PL,Central,Standard Class,Furniture,Furnishings,2016,94,-7.12
28,PL,West,Standard Class,Office Supplies,Supplies,2015,40,-5.64
29,PL,Central,First Class,Office Supplies,Binders,2015,39,-3.74
30,PL,Central,Standard Class,Furniture,Bookcases,2016,17,-3.22


In [22]:
pd.read_sql(
"""
    SELECT SUM(Count) FROM (
    SELECT
        "Order ID prefix"
        ,Region
        ,"Ship Mode"
        ,Category
        ,"Sub-Category"
        ,"Order Year"
        ,COUNT("Order ID") AS Count
        ,AVG("Profit") AS average
    FROM data
    WHERE
        "Order ID prefix" IN (
            --'SL'
            'PL'
        )
    GROUP BY
        "Order ID prefix"
        ,Region
        ,"Ship Mode"
        ,Category
        ,"Sub-Category"
    HAVING average <= 0
    ORDER BY average ASC
    )
""",
conn
)

,SUM(Count)
0,744


In [23]:
pd.read_sql(
"""
    SELECT
        "Order Year"
        ,"Order Month"
        ,COUNT("Order ID") AS Count
        ,SUM("Sales") AS Sales
    FROM data
    GROUP BY
        "Order Year"
        ,"Order Month"
""",
conn
)

,Order Year,Order Month,Count,Sales
0,2014,01,79,14236.89
1,2014,02,46,4519.89
2,2014,03,157,55691.01
3,2014,04,135,28295.34
4,2014,05,122,23648.29
...,...,...,...,...
43,2017,08,218,63120.89
44,2017,09,459,87866.65
45,2017,10,298,77776.92
46,2017,11,459,118447.83
